In [36]:
#函数定义与库的引进
import pandas as pd
import numpy as np
from linearmodels import RandomEffects
from linearmodels.panel import PanelOLS
from linearmodels.iv import IV2SLS
import statsmodels.formula.api as smf
import statsmodels as sm

def show_one(result,title=""):
    df0=pd.DataFrame(columns=["变量",title])
    list1=list(range(1,len(result.params)))+[0]
    for i in list1:
        beta=str(round(result.params.iloc[i],3))
        p=result.pvalues.iloc[i]
        star=""
        if p <= 0.01:
            star="***"
        elif p <= 0.05:
            star="**"
        elif  p <= 0.1:
            star="*"
        if hasattr(result,'bse'):
            bse="("+str(round(result.bse.iloc[i],3))+")"
        elif hasattr(result,'std_errors'):
            bse="("+str(round(result.std_errors.iloc[i],3))+")"
        df0.loc[df0.shape[0]+1]={"变量":result.params.index[i],title:beta+star+""+bse}
    if hasattr(result,'resid'):
        lens=len(result.resid)
    elif hasattr(result,'resids'):
        lens=len(result.resids)
    df0.loc[df0.shape[0]+1]={"变量":"样本量",title:lens}
    df0.loc[df0.shape[0]+1]={"变量":"R平方",title:round(result.rsquared,3)}
    #df0.loc[df0.shape[0]+1]={"变量":"x ",title:"x"}
    return df0.set_index("变量")

def show_more(rs):
    df0=pd.DataFrame()
    for i in range(len(rs)):
        df1=show_one(rs[i],"("+str(i+1)+")")
        df0=pd.concat([df0,df1],axis=1,sort=False)
    df0=df0.fillna("")
    df1=df0.drop(["样本量","R平方","常数项"])
    df1.loc["常数项",:]=df0.loc["常数项",:]
    df1.loc["样本量",:]=df0.loc["样本量",:]
    df1.loc["R平方",:]=df0.loc["R平方",:]    
    return df1 

## 数据清洗

In [ ]:
def old(group):
    total_count = len(group)
    above_65_count = len(group[group['age'] > 60])
    return above_65_count / total_count 

def child(group):
    total_count = len(group)
    above_65_count = len(group[group['age'] < 16])
    return above_65_count / total_count 

def edu(x):
    if x=='文盲/半文盲':
        return 0
    elif x=='小学':
        return 6
    elif x=='初中':
        return 9
    elif x=='高中/中专/技校/职高':
        return 12
    elif x=='大专':
        return 15
    elif x=='大学本科':
        return 16
    elif x=='硕士':
        return 19
    elif x=='博士':
        return 21
def shangye(x):
    if  x in ['0','不知道']:
        return 0
    else :
        return 1
df12 = pd.read_excel('原始数据/2012.xlsx')
df12 = df12[['家庭样本编码','省国标码','户主','是否有自住房','是否持有金融产品','家庭银行贷款额','家庭人口数','现金和存款总值-元-最终版','家庭纯收入','城乡','家庭消费结构','expense','food','dress','house','日用品','医疗','交通通讯','文娱','其他消费','商业保险']]
df12.columns = ['12家户号','省份','户主编号','是否自有住房','是否持有金融产品','家庭银行贷款额','家庭的人口规模','家庭现金及存款总额','总收入','城乡','家庭消费结构','expense','food','dress','house','日用品','医疗','交通通讯','文娱','其他消费','商业保险']
df12 = df12[df12['户主编号']!='不知道']
df12['户主编号'] = df12['户主编号'].apply(lambda x:'pid_a_' + x[-2])
df12['食物占比']=df12.food/df12.expense
df12['衣着占比']=df12.dress/df12.expense
df12['日用品占比']=df12.日用品/df12.expense
df12['医疗占比']=df12.医疗/df12.expense
df12['交通通讯占比']=df12.交通通讯/df12.expense
df12['其他消费占比']=df12.其他消费/df12.expense
df12['文娱占比']=df12.文娱/df12.expense
df12['住房占比']=df12.house/df12.expense
df12=df12.drop(['expense','food','dress','house','日用品','医疗','交通通讯','文娱','其他消费'],axis=1)    
df12h = pd.read_csv(r"原始数据/2012huzhu.csv")
df12h.rename(columns={'fid12': '12家户号'}, inplace=True)    
df12a = pd.merge(df12,df12h,on='12家户号',how='inner')
df12a = df12a[df12a['户主编号'] != 'pid_a_0']
df12a['户主编号'] = df12a.apply(lambda row: row[row['户主编号']], axis=1)
df12a = df12a[['12家户号','省份','户主编号','是否自有住房','是否持有金融产品','家庭银行贷款额','家庭的人口规模','家庭现金及存款总额','总收入','城乡','家庭消费结构','食物占比','衣着占比','住房占比','日用品占比','医疗占比','交通通讯占比','文娱占比','其他消费占比','商业保险']]
df12p = pd.read_csv("原始数据/2012adult.csv")
df12edu = pd.read_csv("原始数据/2012edu.csv")
df12edu = df12edu[['pid','cfps2012_latest_edu']]
df12p = pd.merge(df12p,df12edu,on='pid',how='inner')    
df12p['qe104'] = df12p['qe104'].apply(lambda x:0 if x=='未婚' or x=='同居' else 1)
df12p['qq301'] = df12p['qq301'].apply(lambda x:0 if x=='否' else 1)
df12p['cfps2012_latest_edu'] = df12p['cfps2012_latest_edu'].apply(edu)
df12p = df12p[['pid','fid12','cfps2012_gender','cfps2012_age','qe104','qz202','cfps2012_latest_edu','qq301']]
df12p.columns = ['个人编号','12家户号','性别','年龄','婚姻状况','健康状况','受教育程度','是否经常喝酒']    
df12a['people'] = df12a['12家户号'].apply(str) + df12a['户主编号'].apply(str)
df12p['people'] = df12p['12家户号'].apply(str) + df12p['个人编号'].apply(str)
df12a = pd.merge(df12a,df12p,on='people',how='inner')    
df12age = pd.read_csv("原始数据/2012age.csv")
df12age = df12age[(df12age['tb1b_a_p'] != '不适用') & (df12age['tb1b_a_p'] != '缺失') & (df12age['tb1b_a_p'] != '不知道')]
df12age.columns = ['12家户号','age']
df12age['age'] = df12age['age'].apply(int)
df1 = df12age.groupby('12家户号').apply(old)
df2 = df12age.groupby('12家户号').apply(child)
df12r = pd.concat([df1,df2],axis=1)
df12r = df12r.reset_index()
df12r.columns = ['12家户号','老年抚养比例','少儿抚养比例']
df12a.rename(columns={'12家户号_x': '12家户号'}, inplace=True)
df12u = pd.merge(df12a,df12r,on='12家户号',how='inner') 
df12u['商业保险'] = df12u['商业保险'].apply(shangye)   
df14 = pd.read_csv("原始数据/2014family.csv",encoding='gbk')
df14 = df14.dropna()
df14 = df14[(df14['ft1'] != '不知道') & (df14['urban14'] != '缺失')]
df14['fq2'] = df14['fq2'].apply(lambda x : 1 if x =='家庭成员拥有完全产权' else 0)
df14['ft201'] = df14['ft201'].apply(lambda x: 0 if x=='不适用'  else 1)
df14['ft501'] = df14['ft501'].apply(lambda x: 0 if x=='不适用'  else x)
df14['家庭消费结构'] = 1 - (df14['dress'] +df14['food'] + df14['house'])/df14['expense']
df14  = df14[['fid14', 'fid12', 'provcd14', 'fresp1pid','fq2','ft201','ft501','familysize','savings','fincome2','urban14','家庭消费结构','expense','food','dress','house','日用品','医疗','交通通讯','文娱','其他消费','商业保险']]
df14.columns = ['14家户号','12家户号','省份','户主编号','是否自有住房','是否持有金融产品','家庭银行贷款额','家庭的人口规模','家庭现金及存款总额','总收入','城乡','家庭消费结构','expense','food','dress','house','日用品','医疗','交通通讯','文娱','其他消费','商业保险']
df14['食物占比']=df14.food/df14.expense
df14['衣着占比']=df14.dress/df14.expense
df14['日用品占比']=df14.日用品/df14.expense
df14['医疗占比']=df14.医疗/df14.expense
df14['交通通讯占比']=df14.交通通讯/df14.expense
df14['其他消费占比']=df14.其他消费/df14.expense
df14['文娱占比']=df14.文娱/df14.expense
df14['住房占比']=df14.house/df14.expense
df14=df14.drop(['expense','food','dress','house','日用品','医疗','交通通讯','文娱','其他消费'],axis=1)    
df14['商业保险']=df14['商业保险'].apply(shangye)
df14p = pd.read_csv("原始数据/2014people.csv")
df14p = df14p.drop('pw1r',axis=1)
df14p = df14p.dropna()
df14p['age'] = df14p['cfps_birthy'].apply(lambda x: 2014-x)
df14p['cfps2012_marriage'] = df14p['cfps2012_marriage'].apply(lambda x: 0 if x =='未婚' else 1)
df14p['喝酒'] = df14p['qq301'].apply(lambda x: 0 if x =='否' else 1)
df14p = df14p[['pid','fid14','cfps_gender','age','cfps2012_marriage','qz202','te4','喝酒']]
df14p.columns = ['个人编号','14家户号','性别','年龄','婚姻状况','健康状况','受教育程度','是否经常喝酒']    
df14['people'] = df14['14家户号'].apply(str) + df14['户主编号'].apply(str)
df14p['people'] = df14p['14家户号'].apply(str) + df14p['个人编号'].apply(str)
df14a = pd.merge(df14, df14p, on='people', how='inner')    
df14age = pd.read_csv("原始数据/2014age.csv")
df14age = df14age[(df14age['tb1y_a_p'] != '不适用') & (df14age['tb1y_a_p'] != '缺失') & (df14age['tb1y_a_p'] != '不知道')]
df14age.columns = ['14家户号','age']
df14age['age'] = df14age['age'].apply(int)
df14age['age'] = df14age['age'].apply(lambda x:2014-x)
df1 = df14age.groupby('14家户号').apply(old)
df2 = df14age.groupby('14家户号').apply(child)
df14r = pd.concat([df1,df2],axis=1)
df14r = df14r.reset_index()
df14r.columns = ['14家户号','老年抚养比例','少儿抚养比例']
df14a.rename(columns={'14家户号_x': '14家户号'}, inplace=True)
df14u = pd.merge(df14a,df14r,on='14家户号',how='inner')    
df12edu.columns = ['个人编号','edu']
df14edu = pd.merge(df14u,df12edu,on='个人编号',how='inner')
df14u = df14edu.drop('受教育程度',axis=1)
df14u['edu'] = df14u['edu'].apply(edu)
df14u = df14u.dropna()
df14u.rename(columns={'edu': '受教育程度'}, inplace=True)    
df16 = pd.read_csv("原始数据/2016family.csv",encoding='gbk')
df16 = df16.dropna()
df16 = df16[(df16['urban16'] != '缺失')]
df16['fq2'] = df16['fq2'].apply(lambda x : 1 if x =='家庭成员拥有完全产权' else 0)
df16['ft200'] = df16['ft200'].apply(lambda x: 0 if x=='否'  else 1)
df16['ft501'] = df16['ft501'].apply(lambda x: 0 if x=='不适用'  else x)
df16['家庭消费结构'] = 1 - (df16['dress'] +df16['food'] + df16['house'])/df16['expense']
df16  = df16[['fid16', 'fid12', 'provcd16', 'resp1pid','fq2','ft200','ft501','familysize16','savings','fincome2','urban16','家庭消费结构','expense','food','dress','house','日用品','医疗','交通通讯','文娱','其他消费','商业保险']]
df16.columns = ['16家户号','12家户号','省份','户主编号','是否自有住房','是否持有金融产品','家庭银行贷款额','家庭的人口规模','家庭现金及存款总额','总收入','城乡','家庭消费结构','expense','food','dress','house','日用品','医疗','交通通讯','文娱','其他消费','商业保险']
df16['食物占比']=df16.food/df16.expense
df16['衣着占比']=df16.dress/df16.expense
df16['日用品占比']=df16.日用品/df16.expense
df16['医疗占比']=df16.医疗/df16.expense
df16['交通通讯占比']=df16.交通通讯/df16.expense
df16['其他消费占比']=df16.其他消费/df16.expense
df16['文娱占比']=df16.文娱/df16.expense
df16['住房占比']=df16.house/df16.expense
df16=df16.drop(['expense','food','dress','house','日用品','医疗','交通通讯','文娱','其他消费'],axis=1)    
df16['商业保险']=df16['商业保险'].apply(shangye)
df16p = pd.read_csv("原始数据/2016people.csv")
df16p = df16p.dropna()
df16p['qea0'] = df16p['qea0'].apply(lambda x:0 if x=='未婚' or x=='同居' else 1)
df16p['qq301'] = df16p['qq301'].apply(lambda x:0 if x=='否' else 1)
df16p['cfps2016edu'] = df16p['cfps2016edu'].apply(edu)
df16p = df16p[['pid','fid16','cfps_gender','cfps_age','qea0','qz202','cfps2016edu','qq301']]
df16p.columns = ['个人编号','16家户号','性别','年龄','婚姻状况','健康状况','受教育程度','是否经常喝酒']    
df16['people'] = df16['16家户号'].apply(str) + df16['户主编号'].apply(str)
df16p['people'] = df16p['16家户号'].apply(str) + df16p['个人编号'].apply(str)
df16a = pd.merge(df16, df16p, on='people', how='inner')    
df16age = pd.read_csv("原始数据/2016age.csv")
df16age = df16age.dropna()
df16age = df16age[(df16age['tb1y_a_p'] != '不适用') & (df16age['tb1y_a_p'] != '缺失') & (df16age['tb1y_a_p'] != '不知道')]
df16age.columns = ['16家户号','age']
df16age['age'] = df16age['age'].apply(int)
df16age['age'] = df16age['age'].apply(lambda x:2016-x)
df1 = df16age.groupby('16家户号').apply(old)
df2 = df16age.groupby('16家户号').apply(child)
df16r = pd.concat([df1,df2],axis=1)
df16r = df16r.reset_index()
df16r.columns = ['16家户号','老年抚养比例','少儿抚养比例']
df16a.rename(columns={'16家户号_x': '16家户号'}, inplace=True)
df16u = pd.merge(df16a,df16r,on='16家户号',how='inner')
df16u = df16u[df16u['健康状况'] != '不适用']    
df18 = pd.read_csv("2018family.csv",encoding='gbk')
df18 = df18.dropna()
df18 = df18[(df18['urban18'] != '缺失')]
df18['fq2'] = df18['fq2'].apply(lambda x : 1 if x =='家庭成员拥有完全产权' else 0)
df18['ft200'] = df18['ft200'].apply(lambda x: 0 if x=='否'  else 1)
df18['ft501'] = df18['ft501'].apply(lambda x: 0 if x=='不适用'  else x)
df18['家庭消费结构'] = 1 - (df18['dress'] +df18['food'] + df18['house'])/df18['expense']
df18  = df18[['fid18', 'fid12', 'provcd18', 'resp1pid','fq2','ft200','ft501','familysize18','savings','fincome2','urban18','家庭消费结构','expense','food','dress','house','日用品','医疗','交通通讯','文娱','其他消费','商业保险']]

df18.columns = ['18家户号','12家户号','省份','户主编号','是否自有住房','是否持有金融产品','家庭银行贷款额','家庭的人口规模','家庭现金及存款总额','总收入','城乡','家庭消费结构','expense','food','dress','house','日用品','医疗','交通通讯','文娱','其他消费','商业保险']
df18['食物占比']=df18.food/df18.expense
df18['衣着占比']=df18.dress/df18.expense
df18['日用品占比']=df18.日用品/df18.expense
df18['医疗占比']=df18.医疗/df18.expense
df18['交通通讯占比']=df18.交通通讯/df18.expense
df18['其他消费占比']=df18.其他消费/df18.expense
df18['文娱占比']=df18.文娱/df18.expense
df18['住房占比']=df18.house/df18.expense
df18=df18.drop(['expense','food','dress','house','日用品','医疗','交通通讯','文娱','其他消费'],axis=1)    
df18['商业保险']=df18['商业保险'].apply(shangye)
df18p = pd.read_csv("原始数据/2018people.csv")
df18p = df18p.dropna()
df18p['qea0'] = df18p['qea0'].apply(lambda x:0 if x=='未婚' or x=='同居' else 1)
df18p['qq301'] = df18p['qq301'].apply(lambda x:0 if x=='否' else 1)
df18p['cfps2018edu'] = df18p['cfps2018edu'].apply(edu)
df18p = df18p[df18p['kz202'] != '不适用']
df18p = df18p[['pid','fid18','gender_update','age','qea0','kz202','cfps2018edu','qq301']]
df18p.columns = ['个人编号','18家户号','性别','年龄','婚姻状况','健康状况','受教育程度','是否经常喝酒']    
df18['people'] = df18['18家户号'].apply(str) + df18['户主编号'].apply(str)
df18p['people'] = df18p['18家户号'].apply(str) + df18p['个人编号'].apply(str)
df18a = pd.merge(df18, df18p, on='people', how='inner')    
df18age = pd.read_csv("原始数据/2018age.csv")
df18age = df18age.dropna()
df18age = df18age[(df18age['tb1y_a_p'] != '不适用') & (df18age['tb1y_a_p'] != '缺失') & (df18age['tb1y_a_p'] != '不知道')]
df18age.columns = ['18家户号','age']
df18age['age'] = df18age['age'].apply(int)
df18age['age'] = df18age['age'].apply(lambda x:2018-x)
df1 = df18age.groupby('18家户号').apply(old)
df2 = df18age.groupby('18家户号').apply(child)
df18r = pd.concat([df1,df2],axis=1)
df18r = df18r.reset_index()
df18r.columns = ['18家户号','老年抚养比例','少儿抚养比例']
df18a.rename(columns={'18家户号_x': '18家户号'}, inplace=True)
df18u = pd.merge(df18a,df18r,on='18家户号',how='inner')    
dfu_list = [df12u, df14u, df16u, df18u]
final_dfu = pd.DataFrame(columns=['家户号', '12家户号', '省份', '户主编号', '是否自有住房', '是否持有金融产品', '家庭银行贷款额',
                   '家庭的人口规模', '家庭现金及存款总额', '总收入', '城乡', '家庭消费结构', '性别', '年龄', '婚姻状况',
                   '健康状况', '受教育程度', '是否经常喝酒', '老年抚养比例', '少儿抚养比例', 'year', '食物占比', '衣着占比', '日用品占比',
                   '医疗占比', '交通通讯占比', '其他消费占比', '文娱占比', '住房占比','商业保险'])

for i in range(12, 19, 2):
    dfu = dfu_list.pop(0)

    # Selecting relevant columns for each DataFrame
    dfu = dfu[[f'{i}家户号', '12家户号', '省份', '户主编号', '是否自有住房', '是否持有金融产品', '家庭银行贷款额',
               '家庭的人口规模', '家庭现金及存款总额', '总收入', '城乡', '家庭消费结构', 
               '性别', '年龄', '婚姻状况', '健康状况', '受教育程度', '是否经常喝酒', '老年抚养比例',
               '少儿抚养比例', '食物占比', '衣着占比', '日用品占比', '医疗占比', '交通通讯占比', '其他消费占比', '文娱占比', '住房占比','商业保险']]

    # Adding the 'year' column to the DataFrame
    dfu['year'] = '20'+str(i)

    # Renaming columns
    dfu.columns = ['家户号', '12家户号', '省份', '户主编号', '是否自有住房', '是否持有金融产品', '家庭银行贷款额',
                   '家庭的人口规模', '家庭现金及存款总额', '总收入', '城乡', '家庭消费结构', '性别', '年龄', '婚姻状况',
                   '健康状况', '受教育程度', '是否经常喝酒', '老年抚养比例', '少儿抚养比例', '食物占比', '衣着占比', '日用品占比',
                   '医疗占比', '交通通讯占比', '其他消费占比', '文娱占比', '住房占比','商业保险', 'year']

    # Concatenating the processed DataFrame
    final_dfu = pd.concat([final_dfu, dfu])    
final_dfu = final_dfu[final_dfu['家庭消费结构'] > 0]
final_dfu = final_dfu[final_dfu['家庭现金及存款总额'] > 10]
final_dfu = final_dfu.reset_index(drop=True)    
gdp = pd.read_excel(r"原始数据\gdp.xlsx")
gdp = gdp[['行政区划','总指数','覆盖广度','使用深度','数字化程度','year','人均GDP','贷款/GDP','到杭州的球面距离']]
df = final_dfu
gdp['hecheng'] = gdp['行政区划'].apply(str) + gdp['year'].apply(str)
df['hecheng'] = df['省份'].apply(str) + df['year'].apply(str)
dfu = pd.merge(df,gdp,on='hecheng',how='inner')
dfu = dfu[['家户号', '12家户号', '省份', '户主编号', '是否自有住房', '是否持有金融产品', '家庭银行贷款额',
       '家庭的人口规模', '家庭现金及存款总额', '总收入', '城乡', '家庭消费结构', '性别', '年龄', '婚姻状况',
       '健康状况', '受教育程度', '是否经常喝酒', '老年抚养比例', '少儿抚养比例', 'year_x',
        '总指数', '覆盖广度', '使用深度', '数字化程度', '人均GDP', '贷款/GDP',       
        '到杭州的球面距离','食物占比', '衣着占比', '日用品占比',
         '医疗占比', '交通通讯占比', '其他消费占比', '文娱占比', '住房占比','商业保险']]
dfu.columns = ['家户号', '12家户号', '省份', '户主编号', '是否自有住房', '是否持有金融产品', '家庭银行贷款额',
       '家庭的人口规模', '家庭现金及存款总额', '总收入', '城乡', '家庭消费结构', '性别', '年龄', '婚姻状况',
       '健康状况', '受教育程度', '是否经常喝酒', '老年抚养比例', '少儿抚养比例', 'year',
        '总指数', '覆盖广度', '使用深度', '数字化程度', '人均GDP', '贷款/GDP',
       '到杭州的球面距离','食物占比', '衣着占比', '日用品占比',
        '医疗占比', '交通通讯占比', '其他消费占比', '文娱占比', '住房占比','商业保险']    
dffid = pd.read_csv(r"原始数据\fid.csv")    
df1 = dfu[dfu['year'] == '2012']
dffid1 = dffid[['fid18','fid12']]
dffid1.columns =['18家户号','家户号']
df1 = pd.merge(df1,dffid1,on='家户号',how='inner')
df = df1    
for i in range(14, 19, 2):
    df1 = dfu[dfu['year'] == '20'+str(i)]
    dffid1 = dffid[['fid18',f'fid{i}']]
    dffid1.columns =['18家户号','家户号']
    df1 = pd.merge(df1,dffid1,on='家户号',how='inner')
    df = pd.concat([df,df1])    
df = df[df['家庭银行贷款额'] != '不知道']  

df2012=df[df.year=='2012']
l1=[]
l2=list(df2012.省份)
for i in l2:
    if i not in l1:
        l1.append(i)
df2=pd.DataFrame()
for i in l1:
    df3=df2012[df2012.省份==i].head(1)
    df2=pd.concat([df2,df3])
df4=pd.DataFrame()
for i in l1:
    df3=df2012[df2012.省份==i]
    df5=df2[df2.省份!=i]
    df3['时间变化效应工具变量']=df3.到杭州的球面距离*df5['数字化程度'].mean()
    df4=pd.concat([df4,df3])
    
    
df2014=df[df.year=='2014']
l1=[]
l2=list(df2014.省份)
for i in l2:
    if i not in l1:
        l1.append(i)
df2=pd.DataFrame()
for i in l1:
    df3=df2014[df2014.省份==i].head(1)
    df2=pd.concat([df2,df3])

for i in l1:
    df3=df2014[df2014.省份==i]
    df5=df2[df2.省份!=i]
    df3['时间变化效应工具变量']=df3.到杭州的球面距离*df5['数字化程度'].mean()
    df4=pd.concat([df4,df3])
    
    
df2016=df[df.year=='2016']
l1=[]
l2=list(df2016.省份)
for i in l2:
    if i not in l1:
        l1.append(i)
df2=pd.DataFrame()
for i in l1:
    df3=df2016[df2016.省份==i].head(1)
    df2=pd.concat([df2,df3])

for i in l1:
    df3=df2016[df2016.省份==i]
    df5=df2[df2.省份!=i]
    df3['时间变化效应工具变量']=df3.到杭州的球面距离*df5['数字化程度'].mean()
    df4=pd.concat([df4,df3])
    
df2018=df[df.year=='2018']
l1=[]
l2=list(df2018.省份)
for i in l2:
    if i not in l1:
        l1.append(i)

for i in l1:
    df3=df2018[df2018.省份==i].head(1)
    df2=pd.concat([df2,df3])

for i in l1:
    df3=df2018[df2018.省份==i]
    df5=df2[df2.省份!=i]
    df3['时间变化效应工具变量']=df3.到杭州的球面距离*df5['数字化程度'].mean()
    df4=pd.concat([df4,df3])    
df=df4    
df['1']=df['家户号'].apply(str)+df['12家户号'].apply(str)+df['家庭现金及存款总额'].apply(str)+df['总收入'].apply(str)
l1=list(set(list(df['1'])))    
df1=pd.DataFrame()
for i in l1:
    df2=df[df['1']==i].head(1)
    df1=pd.concat([df1,df2])    
df1=df1[['家户号','省份', '户主编号', '是否自有住房', '是否持有金融产品', '家庭银行贷款额',
       '家庭的人口规模', '家庭现金及存款总额', '总收入', '城乡', '家庭消费结构', '性别', '年龄', '婚姻状况',
       '健康状况', '受教育程度', '是否经常喝酒', '老年抚养比例', '少儿抚养比例', 'year', '总指数', '覆盖广度',
       '使用深度', '数字化程度', '人均GDP', '贷款/GDP', '到杭州的球面距离', '食物占比', '衣着占比', '日用品占比',
       '医疗占比', '交通通讯占比', '其他消费占比', '文娱占比', '住房占比','商业保险', '时间变化效应工具变量']]

In [ ]:
df1.to_excel(r"总数据0.xlsx")
df1

In [ ]:
#数据的简单处理
df=pd.read_excel(r'总数据.xlsx')
df['户主性别']=0
for i in range(len(df)):
    if df.loc[i,'性别']=='男':
        df.loc[i,'户主性别']=1 
    if df.loc[i,'家庭银行贷款额']<=0:
        df.loc[i,'家庭银行贷款额']=df['家庭银行贷款额'].mean()
    if df.loc[i,'总收入']<=0:
        df.loc[i,'总收入']=df['总收入'].mean()

df['户主年龄平方']=df['年龄']*df['年龄']
df['数字普惠金融指数对数']=np.log(df['总指数'])
df['家庭总收入对数']=np.log(df['总收入'])
df['家庭现金及存款对数']=np.log(df['家庭现金及存款总额'])
df['银行贷款额对数']=np.log(df['家庭银行贷款额'])
df['人均GDP对数']=np.log(df['人均GDP'])
df.rename(columns={'贷款/GDP':'贷款GDP之比'}, inplace=True)
df.drop(df[df['健康状况']=='不适用'].index,inplace=True) 
df.drop(df[df['健康状况']=='缺失'].index,inplace=True) 
df=df.set_index(['家户号','year'])
df.to_excel(r"总数据.xlsx")

## 基准回归

为避免遗漏不随时间变化的变量、缓解家庭异质性导致的内生性问题，本文控制了时间和家庭变量，构建双向固定效应模型来研究数字普惠金融对于家庭消费结构的影响。模型如下：
$$Cs_{ijt}=\beta_0+\beta_1ln(Index_{j,t})+\phi X_{ijt}+\delta_i+\theta_t+\varepsilon_{ijt}.$$
其中，下标i表示第i个家庭，j表示省份，t表示时间；$Cs_{ijt}$表示家庭i在j省份第t年的消费结构； $ln(Index_{j,t})$表示第t年j省份的数字普惠金融发展程度；$X_{ijt}$为控制变量;$\delta_i$为家庭的固定效应；$\theta_t$为时间的固定效应；$\varepsilon_{ijt}$为回归残差项。

被解释变量家庭缴费结构为家庭发展和享受型消费支出与八类消费支出总和之比 ，该比例提升可以反映家庭消费结构优化，核心解释变量为数字普惠金融发展指数的对数，选取北京大学数字普惠金融指数；控制变量分为三个层次，一是户主特征变量；二是家庭特征变量，三是地区经济发展变量。

下表为总样本下数字普惠金融对家庭消费结构的基准回归结果，其中第（1）列为只有解释变量、没有控制变量的回归结果；第（2）-（4）列分别为依次加入户主特征变量、家庭特征变量和地区经济发展变量的回归结果。

In [38]:
exog_vars1=['数字普惠金融指数对数']
exog1=df[exog_vars1].copy()
exog1['常数项']=1

exog_vars2=['数字普惠金融指数对数','户主性别', '户主年龄平方', '婚姻状况','健康状况','受教育程度','是否经常喝酒']
exog2=df[exog_vars2].copy()
exog2['常数项']=1

exog_vars3=['数字普惠金融指数对数','户主性别', '户主年龄平方', '婚姻状况','健康状况','受教育程度','是否经常喝酒','家庭总收入对数','家庭的人口规模','老年抚养比例','少儿抚养比例','是否自有住房','是否持有金融产品','家庭现金及存款对数','银行贷款额对数']
exog3=df[exog_vars3].copy()
exog3['常数项']=1

exog_vars4=['数字普惠金融指数对数','户主性别', '户主年龄平方', '婚姻状况','健康状况','受教育程度','是否经常喝酒','家庭总收入对数','家庭的人口规模','老年抚养比例','少儿抚养比例','是否自有住房','是否持有金融产品','家庭现金及存款对数','银行贷款额对数', '人均GDP对数','贷款GDP之比']
exog4=df[exog_vars4].copy()
exog4['常数项']=1

result1=PanelOLS(df['家庭消费结构'],exog1, entity_effects=True,time_effects=True).fit()
result2=PanelOLS(df['家庭消费结构'],exog2, entity_effects=True,time_effects=True).fit()
result3=PanelOLS(df['家庭消费结构'],exog3, entity_effects=True,time_effects=True).fit()
result4=PanelOLS(df['家庭消费结构'],exog4, entity_effects=True,time_effects=True).fit()

re_fixs=[result1,result2,result3,result4]
show_more(re_fixs)

C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\linearmodels\panel\model.py:1214: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\linearmodels\panel\model.py:1214: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\linearmodels\panel\model.py:1214: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


,(1),(2),(3),(4)
变量,,,,
数字普惠金融指数对数,0.272***(0.026),0.27***(0.026),0.264***(0.026),0.222***(0.033)
户主性别,,-0.011**(0.005),-0.01**(0.005),-0.01**(0.005)
户主年龄平方,,-0.0(0.0),-0.0(0.0),-0.0(0.0)
婚姻状况,,-0.02(0.013),-0.025*(0.013),-0.024*(0.013)
健康状况,,-0.001(0.002),-0.001(0.002),-0.002(0.002)
受教育程度,,0.001(0.001),0.001(0.001),0.001(0.001)
是否经常喝酒,,-0.017***(0.006),-0.017***(0.006),-0.016***(0.006)
家庭总收入对数,,,0.001(0.002),0.001(0.002)
家庭的人口规模,,,0.008***(0.002),0.008***(0.002)


根据结果显示：列(1)-(4)的数字普惠金融指数对数的系数均为正且在1%水平下显著，说明数字普惠金融发展对于消费结构升级改善具有更促进作用。列(2)-(4)户主性别的系数均为负且在5%水平下显著，说明女性户主对于消费结构改善升级有促进作用；列(2)-(4)是否经常喝酒的系数均为负且在1%水平下显著，说明喝酒对于消费结构改善升级有负向作用。此外，家庭人口规模、银行贷款数目均有利于消费结构改善升级；而地区人均GDP同样有利于消费结构改善。	

进一步将总体样本分为乡村样本与城镇样本，检验数字普惠金融对家庭消费结构的城乡异质性影响。

In [42]:
df1=df[df['城乡']=='乡村']
df2=df[df['城乡']=='城镇']
df2['健康状况']=df2['健康状况'].apply(int)

exog11=df1[exog_vars1].copy()
exog11['常数项']=1
exog21=df1[exog_vars2].copy()
exog21['常数项']=1
exog31=df1[exog_vars3].copy()
exog31['常数项']=1
exog41=df1[exog_vars4].copy()
exog41['常数项']=1

exog12=df2[exog_vars1].copy()
exog12['常数项']=1
exog22=df2[exog_vars2].copy()
exog22['常数项']=1
exog32=df2[exog_vars3].copy()
exog32['常数项']=1
exog42=df2[exog_vars4].copy()
exog42['常数项']=1

C:\Users\DELL\AppData\Local\Temp\ipykernel_18612\2977885886.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['健康状况']=df2['健康状况'].apply(int)


In [43]:
result11=PanelOLS(df1['家庭消费结构'],exog11, entity_effects=True,time_effects=True).fit()
result21=PanelOLS(df1['家庭消费结构'],exog21, entity_effects=True,time_effects=True).fit()
result31=PanelOLS(df1['家庭消费结构'],exog31, entity_effects=True,time_effects=True).fit()
result41=PanelOLS(df1['家庭消费结构'],exog41, entity_effects=True,time_effects=True).fit()
result12=PanelOLS(df2['家庭消费结构'],exog12, entity_effects=True,time_effects=True).fit()
result22=PanelOLS(df2['家庭消费结构'],exog22, entity_effects=True,time_effects=True).fit()
result32=PanelOLS(df2['家庭消费结构'],exog32, entity_effects=True,time_effects=True).fit()
result42=PanelOLS(df2['家庭消费结构'],exog42, entity_effects=True,time_effects=True).fit()

C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\linearmodels\panel\model.py:1214: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\linearmodels\panel\model.py:1214: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\linearmodels\panel\model.py:1214: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\linearmodels\panel\model.py:1214: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init

下表列(1)-(4)分别为无控制变量和依次加入户主特征变量、家庭特征变量和地区经济发展变量的乡村回归结果值,列(5)-(8)分别为无控制变量和依次加入户主特征变量、家庭特征变量和地区经济发展变量的城镇回归结果值。

根据结果显示，在列(1)-(4)中对乡村进行的回归结果中，数字普惠金融指数对数的系数均在1%水平上显著，且系数大于总体样本回归结果，而城镇样本不显著，由此说明数字普惠金融发展对于乡村的消费结构升级具有更强的促进作用，而对于城镇消费结构升级并无显著作用。

In [44]:
re_fixs1=[result11,result21,result31,result41,result12,result22,result32,result42]
show_more(re_fixs1)

,(1),(2),(3),(4),(5),(6),(7),(8)
变量,,,,,,,,
数字普惠金融指数对数,0.319***(0.042),0.323***(0.042),0.318***(0.042),0.241***(0.054),0.005(0.043),0.004(0.043),-0.003(0.042),0.013(0.053)
户主性别,,-0.013*(0.007),-0.011(0.007),-0.011(0.007),,-0.006(0.007),-0.008(0.007),-0.008(0.007)
户主年龄平方,,-0.0(0.0),-0.0(0.0),-0.0(0.0),,-0.0**(0.0),-0.0(0.0),-0.0(0.0)
婚姻状况,,-0.02(0.019),-0.026(0.019),-0.025(0.019),,-0.023(0.019),-0.03(0.019),-0.03(0.019)
健康状况,,-0.004*(0.002),-0.004*(0.002),-0.004*(0.002),,-0.001(0.002),-0.0(0.002),-0.0(0.002)
受教育程度,,0.0(0.001),0.0(0.001),0.0(0.001),,0.003**(0.001),0.003**(0.001),0.003**(0.001)
是否经常喝酒,,-0.022***(0.008),-0.022***(0.008),-0.021***(0.008),,-0.002(0.009),-0.003(0.009),-0.003(0.009)
家庭总收入对数,,,-0.002(0.002),-0.003(0.002),,,0.002(0.003),0.002(0.003)
家庭的人口规模,,,0.011***(0.003),0.011***(0.003),,,0.01***(0.003),0.01***(0.003)


## 内生性问题解决

尽管本文已经采用双向固定效应模型缓解模型内生性，但由于消费结构与数字普惠金融发展可能存在双向因果关系，即随着消费结构的改善优化，人们的消费需求进一步丰富多样化，直接或间接促进数字普惠金融的发展，从而产生内生性问题。
为解决内生性问题，我们采用工具变量法与滞后一期回归两种方法来缓解内生性问题。

### 工具变量法

在工具变量方法中，本文选取“家庭所在地区与距杭州的球面距离”为工具变量。由于杭州的数字普惠金融发展处于全国领先地位，家庭所在地区距离杭州越近，其受杭州的辐射作用将更强，数字普惠金融的发展程度就越高，满足工具变量的相关性条件；且家庭所在地区与杭州的球面距离并不直接与家庭消费结构相关联，家庭的消费结构并不受与杭州距离的影响，符合工具变量的外生性条件。

而家庭所在地区与杭州的距离为常量，容易导致二阶段估计不生效，因此我们将工具变量与除本省之外的数字普惠金融指数均值相乘，从而最终得到本文所需的具备时间变化效应工具变量。



In [89]:
ivexog_vars=['预测数字普惠金融指数对数','户主性别', '户主年龄平方', '婚姻状况','是否经常喝酒','家庭总收入对数','受教育程度','健康状况','家庭的人口规模','老年抚养比例','少儿抚养比例','是否自有住房','是否持有金融产品','家庭现金及存款对数','银行贷款额对数', '人均GDP对数','贷款GDP之比']

ivresult11 = smf.ols(' 数字普惠金融指数对数 ~ 时间变化效应工具变量 +  户主性别  +户主年龄平方 + 婚姻状况 + 是否经常喝酒 +  家庭总收入对数 + 家庭的人口规模 + 老年抚养比例 + 少儿抚养比例 + 是否自有住房 + 是否持有金融产品 + 家庭现金及存款对数 + 银行贷款额对数 + 人均GDP对数 + 贷款GDP之比', data=df).fit()
df['预测数字普惠金融指数对数'] = ivresult11.predict()
ivexog1=df[ivexog_vars].copy()
ivexog1['常数项']=1
ivresult12=PanelOLS(df['家庭消费结构'],ivexog1, entity_effects=True,time_effects=True).fit()

ivresult21 = smf.ols(' 数字普惠金融指数对数 ~ 时间变化效应工具变量 +  户主性别  +户主年龄平方 + 婚姻状况  + 是否经常喝酒 +  家庭总收入对数 + 家庭的人口规模 + 老年抚养比例 + 少儿抚养比例 + 是否自有住房 + 是否持有金融产品 + 家庭现金及存款对数 + 银行贷款额对数 + 人均GDP对数 + 贷款GDP之比', data=df1).fit()
df1['预测数字普惠金融指数对数'] = ivresult21.predict()
ivexog2=df1[ivexog_vars].copy()
ivexog2['常数项']=1
ivresult22=PanelOLS(df1['家庭消费结构'],ivexog2, entity_effects=True,time_effects=True).fit()

ivresult31 = smf.ols(' 数字普惠金融指数对数 ~ 时间变化效应工具变量 +  户主性别  +户主年龄平方 + 婚姻状况  + 是否经常喝酒 +  家庭总收入对数 + 家庭的人口规模 + 老年抚养比例 + 少儿抚养比例 + 是否自有住房 + 是否持有金融产品 + 家庭现金及存款对数 + 银行贷款额对数 + 人均GDP对数 + 贷款GDP之比', data=df2).fit()
df2['预测数字普惠金融指数对数'] = ivresult31.predict()
ivexog3=df2[ivexog_vars].copy()
ivexog3['常数项']=1
ivresult32=PanelOLS(df2['家庭消费结构'],ivexog3, entity_effects=True,time_effects=True).fit()

C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\linearmodels\panel\model.py:1214: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
C:\Users\DELL\AppData\Local\Temp\ipykernel_18612\2271627660.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['预测数字普惠金融指数对数'] = ivresult21.predict()
C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\linearmodels\panel\model.py:1214: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
C:\Users\DELL\AppData\Local\Temp\ipykernel_18612\2271627660.py:17: Set

下表列(1)-(3)分别为总样本、乡村样本、城镇样本的加入户主特征变量、家庭特征变量和地区经济发展变量的与工具变量结合的双向固定效应模型回归结果值，根据结果显示：总样本中预测数字普惠金融指数对数的系数为正且显著，乡村样本中预测数字普惠金融指数对数的系数为正但不显著，城镇样本中预测数字普惠金融指数对数的系数为正但不显著，考虑到手动两阶段回归的误差问题，城乡样本的不显著结果可接受。因此可以得出数字普惠金融指数对于家庭消费结构有促进作用。

In [90]:
re_fixs2=[ivresult12,ivresult22,ivresult32]
show_more(re_fixs2)

,(1),(2),(3)
变量,,,
户主性别,-0.011**(0.005),-0.011(0.007),-0.008(0.007)
户主年龄平方,-0.0(0.0),-0.0(0.0),-0.0(0.0)
婚姻状况,-0.022*(0.013),-0.02(0.019),-0.03(0.019)
是否经常喝酒,-0.016***(0.006),-0.02**(0.008),-0.003(0.009)
家庭总收入对数,-0.0(0.002),-0.004(0.002),0.002(0.003)
受教育程度,0.001(0.001),0.0(0.001),0.003**(0.001)
健康状况,-0.002(0.002),-0.003(0.002),-0.0(0.002)
家庭的人口规模,0.008***(0.002),0.011***(0.003),0.01***(0.003)
老年抚养比例,-0.007(0.011),-0.029*(0.016),0.009(0.018)


### 滞后一期方法

在滞后一期方法中，下表列(1)-(3)分别为总样本、乡村样本、城镇样本的加入户主特征变量、家庭特征变量和地区经济发展变量的回归结果值，根据结果显示：在克服潜在的内生性问题后，三列回归结果中数字普惠金融指数对数的系数依旧为正且在1%水平下显著，可得出数字普惠金融指数对于家庭消费结构有促进作用。

In [102]:
df = df.sort_values(by="户主编号",ascending=False)
df['数字普惠金融指数对数滞后']=df['数字普惠金融指数对数'].shift(1)
df['家庭消费结构滞后']=df['家庭消费结构'].shift(1)
exog_varszh1=['数字普惠金融指数对数滞后','户主性别', '户主年龄平方', '婚姻状况','健康状况','受教育程度','是否经常喝酒','家庭总收入对数','家庭的人口规模','老年抚养比例','少儿抚养比例','是否自有住房','是否持有金融产品','家庭现金及存款对数','银行贷款额对数', '人均GDP对数','贷款GDP之比']
exogzh1=df[exog_varszh1].copy()
exogzh1['常数项']=1
resultzh1=PanelOLS(df['家庭消费结构滞后'],exogzh1, entity_effects=True,time_effects=True).fit()

df1 = df1.sort_values(by="户主编号",ascending=False)
df1['数字普惠金融指数对数滞后']=df1['数字普惠金融指数对数'].shift(1)
df1['家庭消费结构滞后']=df1['家庭消费结构'].shift(1)
exog_varszh2=['数字普惠金融指数对数滞后','户主性别', '户主年龄平方', '婚姻状况','健康状况','受教育程度','是否经常喝酒','家庭总收入对数','家庭的人口规模','老年抚养比例','少儿抚养比例','是否自有住房','是否持有金融产品','家庭现金及存款对数','银行贷款额对数', '人均GDP对数','贷款GDP之比']
exogzh2=df1[exog_varszh2].copy()
exogzh2['常数项']=1
resultzh2=PanelOLS(df1['家庭消费结构滞后'],exogzh2, entity_effects=True,time_effects=True).fit()

df2 = df2.sort_values(by="户主编号",ascending=False)
df2['数字普惠金融指数对数滞后']=df2['数字普惠金融指数对数'].shift(1)
df2['家庭消费结构滞后']=df2['家庭消费结构'].shift(1)
exog_varszh3=['数字普惠金融指数对数滞后','户主性别', '户主年龄平方', '婚姻状况','健康状况','受教育程度','是否经常喝酒','家庭总收入对数','家庭的人口规模','老年抚养比例','少儿抚养比例','是否自有住房','是否持有金融产品','家庭现金及存款对数','银行贷款额对数', '人均GDP对数','贷款GDP之比']
exogzh3=df2[exog_varszh3].copy()
exogzh3['常数项']=1
resultzh3=PanelOLS(df2['家庭消费结构滞后'],exogzh3, entity_effects=True,time_effects=True).fit()


C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\linearmodels\panel\model.py:1214: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\linearmodels\panel\model.py:1214: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\linearmodels\panel\model.py:1214: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


In [103]:
re_fixszh=[resultzh1,resultzh2,resultzh3]
show_more(re_fixszh)

,(1),(2),(3)
变量,,,
户主性别,0.002(0.005),0.02***(0.007),-0.016**(0.008)
户主年龄平方,-0.0**(0.0),-0.0(0.0),-0.0(0.0)
婚姻状况,0.051***(0.014),0.001(0.02),0.069***(0.021)
健康状况,0.002(0.002),0.003(0.002),0.001(0.003)
受教育程度,-0.002*(0.001),-0.003**(0.001),0.002(0.001)
是否经常喝酒,0.004(0.006),-0.005(0.008),0.01(0.009)
家庭总收入对数,-0.002(0.002),-0.003(0.002),-0.004(0.003)
家庭的人口规模,-0.001(0.002),-0.003(0.003),-0.007*(0.004)
老年抚养比例,0.023*(0.012),0.015(0.017),-0.007(0.019)


## 异质性分析

In [ ]:
dfall=pd.read_excel(r'总数据.xlsx')
dfall['是否东部沿海']=0
dfall['教育情况']=0
dfall['收入水平']=0
for i in range(len(dfall)):
    if dfall.loc[i,'省份']=='江苏省' or dfall.loc[i,'省份']=='山东省' or dfall.loc[i,'省份']=='北京市' \
        or dfall.loc[i,'省份']=='天津市' or dfall.loc[i,'省份']=='浙江省' or dfall.loc[i,'省份']=='福建省' \
            or dfall.loc[i,'省份']=='广东省' or dfall.loc[i,'省份']=='河北省' or dfall.loc[i,'省份']=='上海市':
        dfall.loc[i,'是否东部沿海']=1
    if dfall.loc[i,'受教育程度']>=9 and dfall.loc[i,'受教育程度']<12:
        dfall.loc[i,'教育情况']=1
    elif dfall.loc[i,'受教育程度']>=12:
        dfall.loc[i,'教育情况']=2
    
    if dfall.loc[i,'总收入']>=dfall['总收入'].mean():
        dfall.loc[i,'收入水平']=1
dfall=dfall.dropna()
dfall=dfall.set_index(['year','户主编号'])
dfcity=dfall[dfall['城乡']=='城镇'].copy()
dfcoun =dfall[dfall['城乡']=='乡村'].copy()

In [ ]:
# 分项消费占比分析
## 总数据
div_all=[]
variables=['数字普惠金融指数对数','户主性别','户主年龄平方','婚姻状况','健康状况','受教育程度','是否经常喝酒'\
           ,'家庭的人口规模','总收入','少儿抚养比例','老年抚养比例','是否自有住房','是否持有金融产品','家庭现金及存款总额'\
               ,'家庭银行贷款额','人均GDP对数','贷款GDP之比']
comsum=['食物占比','衣着占比','住房占比','日用品占比','医疗占比','交通通讯占比','文娱占比','其他消费占比']
for i in comsum:
    exog=dfall[variables].copy()
    exog['常数项']=1
    mod = PanelOLS(dfall[i],exog,entity_effects=True,time_effects=True)
    div_all.append( mod.fit())
show_more(div_all)

In [ ]:
##农村
div_coun=[]
for i in comsum:
    exog=dfcoun[variables].copy()
    exog['常数项']=1
    mod = PanelOLS(dfcoun[i],exog,entity_effects=True,time_effects=True)
    div_coun.append( mod.fit())
show_more(div_coun)

In [ ]:
## 城市
div_city=[]
for i in comsum:
    exog=dfcity[variables].copy()
    exog['常数项']=1
    mod = PanelOLS(dfcity[i],exog,entity_effects=True,time_effects=True)
    div_city.append( mod.fit())
show_more(div_city)

In [ ]:
#分区域异质性
div_state=[]

df_all_e=dfall[dfall['是否东部沿海']==1].copy()
df_all_m=dfall[dfall['是否东部沿海']==0].copy()
df_city_e=dfcity[dfcity['是否东部沿海']==1].copy()
df_city_m=dfcity[dfcity['是否东部沿海']==0].copy()
df_coun_e=dfcoun[dfcoun['是否东部沿海']==1].copy()
df_coun_m=dfcoun[dfcoun['是否东部沿海']==0].copy()

for i in [df_all_e,df_all_m,df_city_e,df_city_m,df_coun_e,df_coun_m]:
    exog=i[variables].copy()
    exog['常数项']=1
    mod = PanelOLS(i['家庭消费结构'],exog,entity_effects=True,time_effects=True)
    div_state.append( mod.fit())
show_more(div_state)

In [ ]:
##分家庭收入
div_income=[]

df_all_H=dfall[dfall['收入水平']==1].copy()
df_all_L=dfall[dfall['收入水平']==0].copy()
df_city_H=dfcity[dfcity['收入水平']==1].copy()
df_city_L=dfcity[dfcity['收入水平']==0].copy()
df_coun_H=dfcoun[dfcoun['收入水平']==1].copy()
df_coun_L=dfcoun[dfcoun['收入水平']==0].copy()

for i in [df_all_H,df_all_L,df_city_H,df_city_L,df_coun_H,df_coun_L]:
    exog=i[variables].copy()
    exog['常数项']=1
    mod = PanelOLS(i['家庭消费结构'],exog,entity_effects=True,time_effects=True)
    div_income.append( mod.fit())
show_more(div_income)

## 稳健性检验

In [ ]:
#数据的简单处理
pd.set_option('display.max_columns', 100)
df_1=pd.read_excel("总数据0.xlsx")
# 将商业保险数据写进
newfile=pd.read_excel("总数据.xlsx")[["year","家户号","商业保险"]]
df_1["year"]=df_1['year'].apply(str)
df_1["家户号"]=df_1["家户号"].apply(str)
df_1["uniquecode"]=df_1[["year","家户号"]].apply(''.join, axis=1)
newfile['year']=newfile["year"].apply(str)
newfile["家户号"]=newfile["家户号"].apply(str)
newfile["uniquecode"]=newfile[['year',"家户号"]].apply("".join, axis=1)
newfile['uniquecode']=newfile["uniquecode"].apply(lambda x:x[:-2])
df=df_1.merge(newfile,on="uniquecode")
df.rename(columns={"家户号_x":"家户号","year_x":"year"},inplace=True)
df.drop(["year_y","家户号_y","uniquecode"],axis=1,inplace=True)
df["year"]=df["year"].apply(int)

In [ ]:
def get_log(x):
    return np.log(x) if x>0 else 0
for i in range(len(df)):
    if df.loc[i,'家庭银行贷款额']<=0:
        df.loc[i,'家庭银行贷款额']=df['家庭银行贷款额'].mean()
    if df.loc[i,'总收入']<=0:
        df.loc[i,'总收入']=df['总收入'].mean()
df["覆盖广度对数"]=df["覆盖广度"].apply(get_log)
df["使用深度对数"]=df["使用深度"].apply(get_log)
df["数字化程度对数"]=df["数字化程度"].apply(get_log)
df.rename(columns={'贷款/GDP':'贷款GDP之比'}, inplace=True)
df["户主性别"]=df["性别"].apply(lambda x:int(x=="男"))
df['户主年龄平方']=df['年龄']^2
df['数字普惠金融指数对数']=df['总指数'].apply(get_log)
df['家庭总收入对数']=df['总收入'].apply(get_log)
df['家庭现金及存款对数']=df['家庭现金及存款总额'].apply(get_log)
df['银行贷款额对数']=df['家庭银行贷款额'].apply(get_log)
df['人均GDP对数']=df['人均GDP'].apply(get_log)
df=df[~(df['健康状况']=="不适用") & ~(df["健康状况"]=="缺失")]
df.dropna()
df=df.set_index(["家户号","year"]).copy()

### 替换解释变量
用构成“数字普惠金融总指数”的三个指标依次对其进行替换，分别为“覆盖广度”，“使用深度”和“数字化程度”

In [ ]:
exog1=df[["覆盖广度对数","户主性别","户主年龄平方", "婚姻状况", "健康状况","受教育程度","是否经常喝酒","家庭的人口规模","家庭总收入对数","少儿抚养比例","老年抚养比例","是否自有住房","是否持有金融产品","家庭现金及存款对数","银行贷款额对数","人均GDP对数","贷款GDP之比"]].copy()
exog1["常数项"]=1
exog2=df[["使用深度对数","户主性别","户主年龄平方", "婚姻状况", "健康状况","受教育程度","是否经常喝酒","家庭的人口规模","家庭总收入对数","少儿抚养比例","老年抚养比例","是否自有住房","是否持有金融产品","家庭现金及存款对数","银行贷款额对数","人均GDP对数","贷款GDP之比"]].copy()
exog2["常数项"]=1
exog3=df[["数字化程度对数","户主性别","户主年龄平方", "婚姻状况", "健康状况","受教育程度","是否经常喝酒","家庭的人口规模","家庭总收入对数","少儿抚养比例","老年抚养比例","是否自有住房","是否持有金融产品","家庭现金及存款对数","银行贷款额对数","人均GDP对数","贷款GDP之比"]].copy()
exog3["常数项"]=1
result_1= PanelOLS(df["家庭消费结构"], exog1,time_effects=True,entity_effects=True).fit()
result_2= PanelOLS(df["家庭消费结构"], exog2,time_effects=True,entity_effects=True).fit()
result_3= PanelOLS(df["家庭消费结构"], exog3,time_effects=True,entity_effects=True).fit()
results_1=[result_1,result_2,result_3]
show_more(results_1)

In [ ]:
df1=df[df["城乡"]=="城镇"].copy()
df2=df[df["城乡"]=="乡村"].copy()
# 农村样本(df2)
exog4=df2[["覆盖广度对数","户主性别","户主年龄平方", "婚姻状况", "健康状况","受教育程度","是否经常喝酒","家庭的人口规模","家庭总收入对数","少儿抚养比例","老年抚养比例","是否自有住房","是否持有金融产品","家庭现金及存款对数","银行贷款额对数","人均GDP对数","贷款GDP之比"]].copy()
exog4["常数项"]=1
exog5=df2[["使用深度对数","户主性别","户主年龄平方", "婚姻状况", "健康状况","受教育程度","是否经常喝酒","家庭的人口规模","家庭总收入对数","少儿抚养比例","老年抚养比例","是否自有住房","是否持有金融产品","家庭现金及存款对数","银行贷款额对数","人均GDP对数","贷款GDP之比"]].copy()
exog5["常数项"]=1
exog6=df2[["数字化程度对数","户主性别","户主年龄平方", "婚姻状况", "健康状况","受教育程度","是否经常喝酒","家庭的人口规模","家庭总收入对数","少儿抚养比例","老年抚养比例","是否自有住房","是否持有金融产品","家庭现金及存款对数","银行贷款额对数","人均GDP对数","贷款GDP之比"]].copy()
exog6["常数项"]=1
result_4= PanelOLS(df2["家庭消费结构"], exog4,time_effects=True,entity_effects=True).fit()
result_5= PanelOLS(df2["家庭消费结构"], exog5,time_effects=True,entity_effects=True).fit()
result_6= PanelOLS(df2["家庭消费结构"], exog6,time_effects=True,entity_effects=True).fit()
results_2=[result_4,result_5,result_6]
show_more(results_2)

In [ ]:
# 城市样本(df1)
exog7=df1[["覆盖广度对数","户主性别","户主年龄平方", "婚姻状况", "健康状况","受教育程度","是否经常喝酒","家庭的人口规模","家庭总收入对数","少儿抚养比例","老年抚养比例","是否自有住房","是否持有金融产品","家庭现金及存款对数","银行贷款额对数","人均GDP对数","贷款GDP之比"]].copy()
exog7["常数项"]=1
exog8=df1[["使用深度对数","户主性别","户主年龄平方", "婚姻状况", "健康状况","受教育程度","是否经常喝酒","家庭的人口规模","家庭总收入对数","少儿抚养比例","老年抚养比例","是否自有住房","是否持有金融产品","家庭现金及存款对数","银行贷款额对数","人均GDP对数","贷款GDP之比"]].copy()
exog8["常数项"]=1
exog9=df1[["数字化程度对数","户主性别","户主年龄平方", "婚姻状况", "健康状况","受教育程度","是否经常喝酒","家庭的人口规模","家庭总收入对数","少儿抚养比例","老年抚养比例","是否自有住房","是否持有金融产品","家庭现金及存款对数","银行贷款额对数","人均GDP对数","贷款GDP之比"]].copy()
exog9["常数项"]=1
result_7= PanelOLS(df1["家庭消费结构"], exog7,time_effects=True,entity_effects=True).fit()
result_8= PanelOLS(df1["家庭消费结构"], exog8,time_effects=True,entity_effects=True).fit()
result_9= PanelOLS(df1["家庭消费结构"], exog9,time_effects=True,entity_effects=True).fit()
results_3=[result_7,result_8,result_9]
show_more(results_3)

### 剔除直辖市样本

In [ ]:
df3=df[(df["省份"]!="北京市") & (df["省份"]!="天津市") & (df["省份"]!="上海市") & (df["省份"]!="重庆市")].copy()
df4=df3[df3["城乡"]=="城镇"].copy()
df5=df3[df3["城乡"]=="乡村"].copy()
#总样本
exog10=df3[["数字普惠金融指数对数","户主性别","户主年龄平方", "婚姻状况", "健康状况","受教育程度","是否经常喝酒","家庭的人口规模","家庭总收入对数","少儿抚养比例","老年抚养比例","是否自有住房","是否持有金融产品","家庭现金及存款对数","银行贷款额对数","人均GDP对数","贷款GDP之比"]].copy()
exog10["常数项"]=1
#城市样本
exog11=df4[["数字普惠金融指数对数","户主性别","户主年龄平方", "婚姻状况", "健康状况","受教育程度","是否经常喝酒","家庭的人口规模","家庭总收入对数","少儿抚养比例","老年抚养比例","是否自有住房","是否持有金融产品","家庭现金及存款对数","银行贷款额对数","人均GDP对数","贷款GDP之比"]].copy()
exog11["常数项"]=1
#农村样本
exog12=df5[["数字普惠金融指数对数","户主性别","户主年龄平方", "婚姻状况", "健康状况","受教育程度","是否经常喝酒","家庭的人口规模","家庭总收入对数","少儿抚养比例","老年抚养比例","是否自有住房","是否持有金融产品","家庭现金及存款对数","银行贷款额对数","人均GDP对数","贷款GDP之比"]].copy()
exog12["常数项"]=1
result_10= PanelOLS(df3["家庭消费结构"], exog10,time_effects=True,entity_effects=True).fit()
result_11= PanelOLS(df4["家庭消费结构"], exog11,time_effects=True,entity_effects=True).fit()
result_12= PanelOLS(df5["家庭消费结构"], exog12,time_effects=True,entity_effects=True).fit()
results_4=[result_10,result_11,result_12]
show_more(results_4)

## 影响机制检验

### 信贷约束额度

In [ ]:
信贷约束额度=df["家庭银行贷款额"].mean()
df["信贷约束"]=df['家庭银行贷款额'].apply(lambda x:int(x<信贷约束额度))
df["信贷约束_数字普惠金融指数对数"]=(df["信贷约束"]-df["信贷约束"].mean())*(df["数字普惠金融指数对数"]-df["数字普惠金融指数对数"].mean())
df6=df[df["城乡"]=="城镇"].copy()
df7=df[df["城乡"]=="乡村"].copy()
exog13=df[["数字普惠金融指数对数","信贷约束","信贷约束_数字普惠金融指数对数","户主性别","户主年龄平方", "婚姻状况", "健康状况","受教育程度","是否经常喝酒","家庭的人口规模","家庭总收入对数","少儿抚养比例","老年抚养比例","是否自有住房","是否持有金融产品","家庭现金及存款对数","银行贷款额对数","人均GDP对数","贷款GDP之比"]].copy()
exog13["常数项"]=1
exog14=df6[["数字普惠金融指数对数","信贷约束","信贷约束_数字普惠金融指数对数","户主性别","户主年龄平方", "婚姻状况", "健康状况","受教育程度","是否经常喝酒","家庭的人口规模","家庭总收入对数","少儿抚养比例","老年抚养比例","是否自有住房","是否持有金融产品","家庭现金及存款对数","银行贷款额对数","人均GDP对数","贷款GDP之比"]].copy()
exog14["常数项"]=1
exog15=df7[["数字普惠金融指数对数","信贷约束","信贷约束_数字普惠金融指数对数","户主性别","户主年龄平方", "婚姻状况", "健康状况","受教育程度","是否经常喝酒","家庭的人口规模","家庭总收入对数","少儿抚养比例","老年抚养比例","是否自有住房","是否持有金融产品","家庭现金及存款对数","银行贷款额对数","人均GDP对数","贷款GDP之比"]].copy()
exog15["常数项"]=1
result_13=PanelOLS(df["家庭消费结构"],exog13,time_effects=True,entity_effects=True).fit()
result_14=PanelOLS(df6["家庭消费结构"],exog14,time_effects=True,entity_effects=True).fit()
result_15=PanelOLS(df7["家庭消费结构"],exog15,time_effects=True,entity_effects=True).fit()
results_5=[result_13,result_14,result_15]
show_more(results_5)

### 不确定性约束

In [ ]:
df["不确定性约束"]=df["商业保险"].apply(lambda x:int(x==0))
df["不确定性约束_数字普惠金融指数对数"]=(df["不确定性约束"]-df["不确定性约束"].mean())*(df["数字普惠金融指数对数"]-df["数字普惠金融指数对数"].mean())
df8=df[df["城乡"]=="城镇"].copy()
df9=df[df["城乡"]=="乡村"].copy()
exog16=df[["数字普惠金融指数对数","不确定性约束","不确定性约束_数字普惠金融指数对数","户主性别","户主年龄平方", "婚姻状况", "健康状况","受教育程度","是否经常喝酒","家庭的人口规模","家庭总收入对数","少儿抚养比例","老年抚养比例","是否自有住房","是否持有金融产品","家庭现金及存款对数","银行贷款额对数","人均GDP对数","贷款GDP之比"]].copy()
exog16["常数项"]=1
exog17=df8[["数字普惠金融指数对数","不确定性约束","不确定性约束_数字普惠金融指数对数","户主性别","户主年龄平方", "婚姻状况", "健康状况","受教育程度","是否经常喝酒","家庭的人口规模","家庭总收入对数","少儿抚养比例","老年抚养比例","是否自有住房","是否持有金融产品","家庭现金及存款对数","银行贷款额对数","人均GDP对数","贷款GDP之比"]].copy()
exog17["常数项"]=1
exog18=df9[["数字普惠金融指数对数","不确定性约束","不确定性约束_数字普惠金融指数对数","户主性别","户主年龄平方", "婚姻状况", "健康状况","受教育程度","是否经常喝酒","家庭的人口规模","家庭总收入对数","少儿抚养比例","老年抚养比例","是否自有住房","是否持有金融产品","家庭现金及存款对数","银行贷款额对数","人均GDP对数","贷款GDP之比"]].copy()
exog18["常数项"]=1
result_16=PanelOLS(df["家庭消费结构"],exog16,time_effects=True,entity_effects=True).fit()
result_17=PanelOLS(df8["家庭消费结构"],exog17,time_effects=True,entity_effects=True).fit()
result_18=PanelOLS(df9["家庭消费结构"],exog18,time_effects=True,entity_effects=True).fit()
results_6=[result_16,result_17,result_18]
show_more(results_6)